In [1]:
!ls /kaggle/input/titanic

gender_submission.csv  test.csv  train.csv


## 1. 라이브러리 불러오기

In [2]:
# 필요한 라이브러리를 불러옵니다.

## 데이터 분석 관련
import pandas as pd
# Series, DataFrame을 미리 import 해주면 나중에 반복해서 안쳐줘도 된다
# https://dojang.io/mod/page/view.php?id=2450
from pandas import Series, DataFrame 
import numpy as np

## 데이터 시각화 관련
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid') # set_style 종류 찾아보기

## 그래프 출력에 필요한 IPython 명령어
%matplotlib inline 
# 넣어주지 않으면 그래프, 그림 등이 출력이 되지 않는건가?

## Scikit-Learn의 다양한 머신러닝 모듈을 불러옵니다.
## 분류 알고리즘 중에서 선형회귀, 서포트벡터머신, 랜덤포레스트, K-최근접이웃 알고리즘을 사용해보려고 합니다.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

## 2. 데이터 읽기

In [3]:
# 데이터 가져오기(pd.read_csv())
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")

# import OS
# OS.getenv 현재 코드가 실행되는 파일의 경로 불러온다. 즉, 현재 실행하고 있는 경로에 없는 파일을 불러오는 것
# 절대경로/상대경로의 차이

# 데이터 미리보기
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


- 시사점 : 이름과 티켓 데이터의 불규칙성이 많은 것은 알 수 있음

In [4]:
# 테스트 데이터와 훈련 데이터 info. 확인
train_df.info()
print('-' * 40) # 문자열 연산 가능
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Passenger

In [5]:
# float 은 연속적인 데이터

In [6]:
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
test_df = test_df.drop(['Name', 'Ticket'], axis=1)

## 3. 데이터 처리하기

### 3.1 Pclass
- 1등석, 2등석, 3등석과 같은 정보

In [7]:
train_df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [8]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [9]:
# 범주형 데이터 one-hot-encoding
# Pclass 컬럼을 삭제하고, 범주형 개별 컬럼으로 데이터를 전환함

pclass_train_dummies = pd.get_dummies(train_df['Pclass'])
pclass_test_dummies = pd.get_dummies(test_df['Pclass'])

train_df.drop(['Pclass'], axis=1, inplace=True)
test_df.drop(['Pclass'], axis=1, inplace=True)

# inplace = True 리턴 값을 바로 자기 자신에게 저장해준다

train_df = train_df.join(pclass_train_dummies)
test_df = test_df.join(pclass_test_dummies)

In [10]:
train_df.head()

,Survived,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,1,2,3
0,0,male,22.0,1,0,7.2500,NaN,S,0,0,1
1,1,female,38.0,1,0,71.2833,C85,C,1,0,0
2,1,female,26.0,0,0,7.9250,NaN,S,0,0,1
3,1,female,35.0,1,0,53.1000,C123,S,1,0,0
4,0,male,35.0,0,0,8.0500,NaN,S,0,0,1


### 3.2 Sex
- 성별(male, female)

In [11]:
sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])

# 컬럼명 추가 해줄 수 있음
sex_train_dummies.columns = ['Female', 'Male'] 
sex_test_dummies.columns = ['Female', 'Male']

train_df.drop(['Sex'], axis=1, inplace=True)
test_df.drop(['Sex'], axis=1, inplace=True) 

# join() : inplace 파라메터 X
train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

In [12]:
train_df.head()

,Survived,Age,SibSp,Parch,Fare,Cabin,Embarked,1,2,3,Female,Male
0,0,22.0,1,0,7.2500,NaN,S,0,0,1,0,1
1,1,38.0,1,0,71.2833,C85,C,1,0,0,1,0
2,1,26.0,0,0,7.9250,NaN,S,0,0,1,1,0
3,1,35.0,1,0,53.1000,C123,S,1,0,0,1,0
4,0,35.0,0,0,8.0500,NaN,S,0,0,1,0,1


### 3.3 Age

In [13]:
# 결측치
print(train_df['Age'].isnull().sum(), test_df['Age'].isnull().sum(), sep=', ')

177, 86


In [14]:
# 결측치를 평균값으로 채워주기
train_df['Age'].fillna(train_df['Age'].mean(), inplace=True)
test_df['Age'].fillna(train_df['Age'].mean(), inplace=True)

### 3.4 SibSp & Panch
- 형제, 자매, 부모님 등

### 3.5 Fare
- 탑승료

In [15]:
# 테스트 데이터의 결측치 1개를 무단탑승으로 가정
test_df[test_df['Fare'].isnull()]
test_df["Fare"].fillna(0, inplace=True)

### 3.6 Cabin
- 객실

In [16]:
# 결측치가 많아 drop
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

### 3.7 Embarked
- 탑승항구 위치

In [17]:
train_df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [18]:
test_df['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [19]:
train_df['Embarked'].fillna('S', inplace=True)
test_df['Embarked'].fillna('S', inplace=True)

In [20]:
embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

embarked_train_dummies.columns = ['S', 'C', 'Q']
embarked_test_dummies.columns = ['S', 'C', 'Q']

train_df.drop(['Embarked'], axis=1, inplace=True)
test_df.drop(['Embarked'], axis=1, inplace=True)

train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_test_dummies)

## 4. 데이터 나누기
- 학습용 데이터를 위해 정보, 생존 여부 등과 같은 형태의 데이터를 나눠준다.

In [21]:
X_train = train_df.drop("Survived",axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId",axis=1).copy()
# 왜 copy()를 사용했을까?
# inplace = True가 아니기 때문에 X_train, X_test 코드 차이가 없다.